In [1]:
import numpy as np
import matplotlib.pyplot as plt
import SiPMWaveGen as swg
import scipy.signal as scisig

In [2]:
def FindPedestal(p, m):
    noOutlier = RejectOutliers(p, m=m)
    return np.mean(noOutlier)

In [3]:
def FindDigitizedPedestal(p, m, nBits, dynamicRange):
    noOutlier = FindPedestal(p=p, m=m)
    res = dynamicRange / (2 ** nBits - 1)
    noiseInADC = swg.getRawADC(speAmplitude / 15, res)

    return [np.mean(noOutlier), noiseInADC]


In [4]:
def RejectOutliers(data, m=2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]

In [5]:
def WaveformDiscriminator(p,
                          nNoiseSigmaThreshold=1,
                          sgFilter=True,
                          sgWindow=15,
                          sgPolyOrder=3):
    [baselineVal, noiseInADC] = FindDigitizedPedestal(p=p, m=3, nBits=12, dynamicRange=1)
    if sgFilter:
        filter_p = scisig.savgol_filter(x=p, window_length=sgWindow, polyorder=sgPolyOrder)
        hitLogic = np.array([(True if pi < baselineVal - nNoiseSigmaThreshold * noiseInADC else False) for pi in filter_p])
    else:
        hitLogic = np.array([(True if pi < baselineVal - nNoiseSigmaThreshold * noiseInADC else False) for pi in p])
    return hitLogic


In [14]:
def DiscriminatorConditioning(p,
                              durationTheshold=5,
                              adjDurationThreshold=5,
                              nNoiseSigmaThreshold=1,
                              sgFilter=True,
                              sgWindow=15,
                              sgPolyOrder=3):
    hitLogic = WaveformDiscriminator(p=p, 
                                     nNoiseSigmaThreshold=nNoiseSigmaThreshold, 
                                     sgFilter=sgFilter, 
                                     sgWindow=sgWindow, 
                                     sgPolyOrder=sgPolyOrder)
    
    for i in range(1, np.size(hitLogic)):
        if ((not hitLogic[i-1]) and hitLogic[i]) and hitLogic[i]:
            countDuration = 0
            for j in range(i, np.size(hitLogic)-1):
                if hitLogic[j]:
                    countDuration = countDuration + 1
                if not hitLogic[j+1]:
                    break
                    
            if countDuration < durationTheshold:
                for j in range(i, i + countDuration):
                    hitLogic[j] = False
                    
    for i in range(1, np.size(hitLogic)):
        if (hitLogic[i-1] and (not hitLogic[i])) and (not hitLogic[i]):
            countDuration = 0
            for j in range(i, np.size(hitLogic)-1):
                if (not hitLogic[j]):
                    countDuration = countDuration + 1
                if hitLogic[j+1]:
                    break
                    
            if countDuration < adjDurationThreshold:
                for j in range(i, i + countDuration):
                    hitLogic[j] = True
    
    return hitLogic
        

In [ ]:
def HitFinder(p,
              cfdThreshold=0.2,
              durationTheshold=5,
              adjDurationThreshold=5,
              nNoiseSigmaThreshold=1,
              sgFilter=True,
              sgWindow=15,
              sgPolyOrder=3):
    hitLogic = DiscriminatorConditioning(p=p,
                         durationTheshold=durationTheshold,
                         adjDurationThreshold=adjDurationThreshold,
                         nNoiseSigmaThreshold=nNoiseSigmaThreshold,
                         sgFilter=sgFilter,
                         sgWindow=sgWindow,
                         sgPolyOrder=sgPolyOrder)
    deriv_p = scisig.savgol_filter(p, window_length=sgWindow, polyorder=sgPolyOrder, deriv=1)

In [ ]:
def checkExtremaInWindow(deriv, hitLogic, startIndex):
    countDuration = 0
    for i in range(startIndex, np.size(deriv) - 1):
        countDuration = countDuration + 1
        if not hitLogic[i + 1]:
            break
    
    extremaIndex = startIndex
    for i in range(startIndex, startIndex + countDuration):
        if ():
            extremaIndex = i

In [15]:
timeSample = 0.2 # nanosec
nSamples = 1024 # no. of sampling points in a trigger
speAmplitude = 0.15 # single photon-electron response amplitude in volt, noise sigma is 1/10 of the speAmplitude
riseTime = 1.5 # nanosec
fallTime = 3 # nanosec
nBits = 12 # 12bit ADC
voltMin = -0.9 # minimum voltage ADC in volt
dynamicRange = 1 # dynamic range of ADC
offset = 2000 # offset ADC
for i in range(10):
    [t, p, true_p] = swg.aDigitizedTrigger(dt=timeSample, 
                                           nsamples=nSamples, 
                                           speAmplitude=speAmplitude, 
                                           riseTime=riseTime, 
                                           fallTime=fallTime, 
                                           nBits=nBits, 
                                           voltMin=voltMin, 
                                           dynamicRange=dynamicRange, 
                                           offset=offset)
    
    durationTheshold=10
    adjDurationThreshold=10
    nNoiseSigmaThreshold=1
    sgFilter=True
    sgWindow=15
    sgPolyOrder=3
    hitLogic = DiscriminatorConditioning(p=p,
                         durationTheshold=durationTheshold,
                         adjDurationThreshold=adjDurationThreshold,
                         nNoiseSigmaThreshold=nNoiseSigmaThreshold,
                         sgFilter=sgFilter,
                         sgWindow=sgWindow,
                         sgPolyOrder=sgPolyOrder)
    filter_p = scisig.savgol_filter(x=p, window_length=sgWindow, polyorder=sgPolyOrder)
    
    [baselineVal, noiseADC] = FindDigitizedPedestal(p=p, m=3, nBits=12, dynamicRange=1)
    threshold = np.array([baselineVal - nNoiseSigmaThreshold*noiseADC for ti in t])
    
    plt.plot(t, p, t, filter_p, t, threshold)
    plt.show()

    plt.plot(t, hitLogic)
    plt.show()

In [23]:

a=range(0,5,1)
a[5]

IndexError: range object index out of range